In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tqdm

In [2]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

## 定义需要的类和函数

In [3]:
class ashareeodprice():
    def __init__(self,asharepath='/Users/chenjiayi/Desktop/陈佳懿_相关数据/new_7.23/ashare_nost.csv') -> None:
        self.asharepath=asharepath
        self.ashare_data=None
    
    def read_data(self):
        self.ashare_data = pd.read_csv(self.asharepath, parse_dates=['TRADE_DT'],encoding='gbk')
        # self.data.sort_values('TRADE_DT', inplace=True)

    def get_data(self):
        return self.ashare_data

    def display_data(self):
        print(self.ashare_data)

    def calculate_return_for_stock(self, stock_code, start_date, end_date):
        """筛选特定日期"""
        # 将start_date转为datetime对象
        start_date = pd.to_datetime(start_date)
        # 计算start_date减去90天的日期
        cutoff_date = start_date - timedelta(days=90)

        # 筛选特定股票和日期范围的数据，并剔除新股
        filtered_data = self.ashare_data[
            (self.ashare_data['S_INFO_WINDCODE'] == stock_code) &
            (self.ashare_data['TRADE_DT'] >= start_date) &
            (self.ashare_data['TRADE_DT'] <= end_date) &
            ~((self.ashare_data['S_DQ_TRADESTATUS'] == 'N') & (self.ashare_data['TRADE_DT'] > cutoff_date))
        ]

        """计算特定股票在两个日期之间的收益率"""
        # filtered_data = self.ashare_data[(self.ashare_data['S_INFO_WINDCODE'] == stock_code) &
        #                           (self.ashare_data['TRADE_DT'] >= start_date) &
        #                           (self.ashare_data['TRADE_DT'] <= end_date)]
        if filtered_data.empty:
            return None
        start_price = filtered_data.iloc[0]['S_DQ_ADJCLOSE']
        end_price = filtered_data.iloc[-1]['S_DQ_ADJCLOSE']
        return (end_price - start_price) / start_price
    
    def calculate_returns_for_all_stocks(self, start_date, end_date):
        """计算所有股票在两个日期之间的收益率"""
        stock_codes = self.ashare_data['S_INFO_WINDCODE'].unique()
        returns = []
        for stock_code in stock_codes:
            stock_return = self.calculate_return_for_stock(stock_code, start_date, end_date)
            if stock_return is not None:
                returns.append({'StockCode': stock_code, 'Return': stock_return})
        return pd.DataFrame(returns)
    
    def rank_and_split_returns(self, returns_df):
        """对收益率进行排名并转换为分位数,前50%按升序排列,后50%按降序排列,并按照排名进行加权"""
        sorted_df = returns_df.sort_values(by='Return', ascending=False).copy()
        total_count = len(sorted_df)
        mid_point = total_count // 2

        top_50 = sorted_df.iloc[:mid_point]
        bottom_50 = sorted_df.iloc[mid_point:]

        top_50['Rank'] = top_50['Return'].rank(ascending=True)
        bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
        
        top_50.loc[:,'Weight'] = top_50['Rank'] / (top_50['Rank'].sum() + bottom_50['Rank'].sum())
        bottom_50.loc[:,'Weight']  = bottom_50['Rank'] / (top_50['Rank'].sum() + bottom_50['Rank'].sum())
        top_50=top_50.set_index('StockCode')
        bottom_50=bottom_50.set_index('StockCode')
        sorted_df = pd.concat([top_50, bottom_50])
        sorted_df['Rank'] = sorted_df['Rank'].astype(int)
        return sorted_df, top_50, bottom_50



In [4]:
reader = ashareeodprice()
reader.read_data()
# 计算任意两个日期之间的收益率
start_date = '2023-06-30'  # 修改为你的开始日期
end_date = '2023-07-31'    # 修改为你的结束日期
returns_df = reader.calculate_returns_for_all_stocks(start_date, end_date)
stock_return_rank, top_50, bottom_50 = reader.rank_and_split_returns(returns_df)
stock_return_rank

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

,Return,Rank,Weight
StockCode,,,
000656.SZ,1.287273,2618,0.000382
600280.SH,1.213176,2617,0.000382
301398.SZ,1.019733,2616,0.000382
002146.SZ,0.974636,2615,0.000381
300844.SZ,0.960326,2614,0.000381
...,...,...,...
301368.SZ,-0.360490,2614,0.000381
688498.SH,-0.361391,2615,0.000381
603042.SH,-0.365558,2616,0.000382


In [5]:
#计算rankmatrix
class fundportfolio():
    def __init__(self,portfoliopath='/Users/chenjiayi/Desktop/陈佳懿_相关数据/new_7.23/fund_2303to2403.csv') -> None:
        self.portfoliopath=portfoliopath
        self.portfolio_data=None
        
    def read_data(self):
        self.portfolio_data = pd.read_csv(self.portfoliopath, parse_dates=['F_PRT_ENDDATE'])
        try:
            self.portfolio_data['F_PRT_ENDDATE'] = pd.to_datetime(self.portfolio_data['F_PRT_ENDDATE'], format='%Y%m%d')
        except Exception as e:
            print(f"Error converting F_PRT_ENDDATE to datetime: {e}")
            raise
        self.portfolio_data.sort_values('F_PRT_ENDDATE', inplace=True)

    def get_data(self):
        return self.portfolio_data

    def display_data(self):
        print(self.portfolio_data)
    
    def filter_data_by_enddate(self, enddate_str):
        """根据日期过滤数据"""
        if self.portfolio_data is None:
            raise ValueError("数据未加载，请先调用 read_data() 方法加载数据。")
        if not pd.api.types.is_datetime64_any_dtype(self.portfolio_data['F_PRT_ENDDATE']):
            raise TypeError("F_PRT_ENDDATE 列不是日期类型，请检查数据格式。")
        return self.portfolio_data[self.portfolio_data['F_PRT_ENDDATE'].dt.strftime('%Y-%m-%d') == enddate_str]

    def top_holdings_by_fund(self, fund_code, top_n=10):
        """选出特定基金持仓权重前top_n的股票"""
        fund_data = self.portfolio_data[self.portfolio_data['S_INFO_WINDCODE'] == fund_code]
        top_holdings = fund_data.sort_values(by='F_PRT_STKVALUETONAV', ascending=False).head(top_n)
        return top_holdings
    
    def remove_funds_with_hk_stocks(self):
        """删除持有港股的基金"""
        hk_stocks_funds = self.portfolio_data[self.portfolio_data['S_INFO_STOCKWINDCODE'].str.endswith('HK')]['S_INFO_WINDCODE'].unique()
        self.portfolio_data = self.portfolio_data[~self.portfolio_data['S_INFO_WINDCODE'].isin(hk_stocks_funds)]

    def top_holdings_for_all_funds(self, top_n=10):
        """选出所有基金持仓权重前top_n的股票"""
        self.remove_funds_with_hk_stocks()
        all_top_holdings = pd.DataFrame()
        fund_codes = self.portfolio_data['S_INFO_WINDCODE'].unique()
        for fund_code in fund_codes:
            top_holdings = self.top_holdings_by_fund(fund_code, top_n)
            all_top_holdings = pd.concat([all_top_holdings, top_holdings])
        return all_top_holdings
    
    def filter_and_top_holdings_for_all_funds(self, enddate_str, top_n=10):
        """筛选出指定日期的所有基金持仓权重前top_n的股票"""
        filtered_data = self.filter_data_by_enddate(enddate_str)
        self.portfolio_data = filtered_data  
        return self.top_holdings_for_all_funds(top_n)
    
    def create_rank_matrix(self,rawmatrix,top_50,bottom_50):

        matrix_raw=rawmatrix.pivot(index='S_INFO_STOCKWINDCODE',columns='S_INFO_WINDCODE',values='F_PRT_STKVALUETONAV')
        matrix_raw.fillna(0,inplace=True)
        top_50_id=pd.DataFrame(top_50.index)
        bottom_50_id=pd.DataFrame(bottom_50.index)
        top50_stocks = top_50_id['StockCode'].tolist()
        bottom50_stocks = bottom_50_id['StockCode'].tolist()
        top50_stocks = [stock for stock in top50_stocks if stock in matrix_raw.index]
        top50_df = matrix_raw.loc[top50_stocks]
        bottom50_stocks = [stock for stock in bottom50_stocks if stock in matrix_raw.index]
        bottom50_df = matrix_raw.loc[bottom50_stocks]
        quantile_top50_holdings = top50_df.rank(pct=True,method='average',axis=1)
        quantile_bottom50_holdings=bottom50_df.rank(pct=True,method='average',ascending=False,axis=1)
        rank=pd.concat([quantile_top50_holdings,quantile_bottom50_holdings])
        return rank
        

In [6]:
fundportfolio1=fundportfolio()
fundportfolio1.read_data()
#按日期得到报告期持仓信息
enddate_str = '2023-06-30' 
a = fundportfolio1.filter_and_top_holdings_for_all_funds(enddate_str, top_n=10)


In [7]:
a

,Unnamed: 0,S_INFO_WINDCODE,F_PRT_ENDDATE,S_INFO_STOCKWINDCODE,F_PRT_STKVALUE,F_PRT_STKVALUETONAV,STOCK_PER
889348,889348,015233.OF,2023-06-30,600519.SH,1.285160e+09,9.78,10.41
889344,889344,015233.OF,2023-06-30,000568.SZ,1.153044e+09,8.78,9.34
889346,889346,015233.OF,2023-06-30,000858.SZ,1.079569e+09,8.22,8.75
889349,889349,015233.OF,2023-06-30,600600.SH,1.036300e+09,7.89,8.40
889352,889352,015233.OF,2023-06-30,000333.SZ,8.838520e+08,6.73,7.16
...,...,...,...,...,...,...,...
502716,502716,009359.OF,2023-06-30,300316.SZ,1.240750e+06,1.17,5.91
502725,502725,009359.OF,2023-06-30,300124.SZ,1.072307e+06,1.01,5.11
502720,502720,009359.OF,2023-06-30,600741.SH,1.066988e+06,1.01,5.08
502719,502719,009359.OF,2023-06-30,600690.SH,1.040164e+06,0.98,4.96


In [8]:
b=fundportfolio1.create_rank_matrix(a,top_50,bottom_50)
b

S_INFO_WINDCODE,000001.OF,000003.OF,000004.OF,000006.OF,000008.OF,000011.OF,000014.OF,000017.OF,000020.OF,000021.OF,...,970130.OF,970131.OF,970184.OF,970185.OF,970188.OF,970189.OF,970190.OF,970204.OF,970205.OF,970207.OF
S_INFO_STOCKWINDCODE,,,,,,,,,,,,,,,,,,,,,
000656.SZ,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,...,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062
002146.SZ,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,...,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062
300169.SZ,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
601099.SH,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,...,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062,0.500062
600266.SH,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,...,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647,0.498647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688195.SH,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,...,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554,0.500554
873593.BJ,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,...,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492,0.500492
688498.SH,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,...,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568,0.503568


In [9]:
#计算基金收益率
class fundreturns:
    def __init__(self,fundreturnpath='/Users/chenjiayi/Desktop/陈佳懿_相关数据/new_7.23/ChinaMutualFundNAV.CSV') -> None:
        self.fundreturnpath=fundreturnpath
        self.fundreturn_data=[]
    
    def get_data(self):
        self.fundreturn_data = pd.read_csv(self.fundreturnpath,parse_dates=['PRICE_DATE'])
    
    def calculate_return_for_fund(self, fund_code, start_date, end_date):
        """计算特定基金在报告期的收益率"""
        filtered_data = self.fundreturn_data[(self.fundreturn_data['F_INFO_WINDCODE'] == fund_code) &
                                  (self.fundreturn_data['PRICE_DATE'] >= start_date)&
                                   (self.fundreturn_data['PRICE_DATE']<=end_date) ]
        
        if filtered_data.empty:
            return None
        start_price = filtered_data.iloc[-1]['F_NAV_ADJUSTED']
        end_price = filtered_data.iloc[0]['F_NAV_ADJUSTED']
        return (end_price - start_price) / start_price
    
    # def calculate_returns_for_all_funds(self, report_time):
    #     """计算所有基金在报告期的收益率"""
    #     navchange=[]
    #     fund_codes = self.fundreturn_data['S_INFO_WINDCODE'].unique()
    #     for fund_code in fund_codes:
    #         fund_return = self.calculate_return_for_fund(fund_code, report_time)
    #         if fund_return is not None:
    #             navchange.append({'FundCode': fund_code, 'Fund_return': fund_return})
    #     return pd.DataFrame(navchange)
    
    def calculate_returns_for_all_funds(self, start_date, end_date):
        """计算所有基金在报告期的收益率"""
        navchange=[]
        fund_codes = self.fundreturn_data['F_INFO_WINDCODE'].unique()
        for fund_code in fund_codes:
            fund_return = self.calculate_return_for_fund(fund_code, start_date, end_date)
            if fund_return is not None:
                navchange.append({'FundCode': fund_code, 'Fund_return': fund_return})
        return pd.DataFrame(navchange)
    
    def calculate_return_rank_for_funds(self,start_date, end_date):
        """按照Fund_return升序排序并返回分位数"""
        df_sorted = self.calculate_returns_for_all_funds(start_date, end_date).sort_values(by='Fund_return', ascending=True).reset_index(drop=True)
        # df_sorted['Quantile'] = pd.qcut(df_sorted['Fund_return'], q=100, labels=False,duplicates='drop') / 100
        df_sorted['Quantile'] = df_sorted['Fund_return'].rank(pct=True,method='average')
        return df_sorted   

In [10]:
# ddd['Fund_return'].dtype

In [11]:
#计算stuct因子
def calculatestuct(stock_return_rank,b):
    stock_rank_weight=pd.DataFrame(stock_return_rank['Weight'])
    stock_rank_weight_id= b.index.tolist()
    stock_rank_weight_id_1232= [stock for stock in stock_rank_weight_id if stock in stock_rank_weight.index]
    target = stock_rank_weight.loc[stock_rank_weight_id_1232]
    weighted_holdings_matrix = b.multiply(target['Weight'], axis=0)
    col_sums = weighted_holdings_matrix.sum(axis=0)
    # 添加一行记录每列的和，即计算stuctraw因子
    weighted_holdings_matrix.loc['stuct_raw'] = col_sums
    
    sum_row=weighted_holdings_matrix.loc['stuct_raw']
    max_value = sum_row.max()
    min_value = sum_row.min()
    range_value = max_value - min_value

    # 避免除以零的情况
    if range_value == 0:
        linear_mapped_sum = sum_row - min_value
    else:
        linear_mapped_sum = (sum_row - min_value) / range_value
    weighted_holdings_matrix.loc['stuct'] = linear_mapped_sum
    weighted_holdings_matrix


    
    return weighted_holdings_matrix


In [12]:
#计算dressing或者invisible
def calculate_alpha(return_rank_quantile,weighted_holdings_matrix):
    return_rank_quantile = return_rank_quantile[return_rank_quantile['FundCode'].isin(weighted_holdings_matrix.columns)]
    stuct_row = weighted_holdings_matrix.loc['stuct']

    # 计算 Quantile 列减去 stuct 行的对应值
    return_rank_quantile['Adjusted_Quantile'] = return_rank_quantile.apply(
        lambda row: row['Quantile'] - stuct_row[row['FundCode']], axis=1
    )

    # 将结果添加到持仓矩阵的最后一行
    adjusted_quantile = return_rank_quantile.set_index('FundCode')['Adjusted_Quantile']
    weighted_holdings_matrix.loc['alpha'] = adjusted_quantile
    return weighted_holdings_matrix.dropna(axis=1)


In [13]:
def calculate_rankic(weighted_holdings_matrix,future_return):
    # 提取 adjusted_quantile 行
    adjusted_quantile = weighted_holdings_matrix.loc['alpha']

    # 确保基金代码在收益矩阵和持仓矩阵中一致
    future_return= future_return[future_return['FundCode'].isin(adjusted_quantile.index)]

    # 添加调整后的 Quantile 差值到收益矩阵
    future_return['alpha'] =future_return['FundCode'].map(adjusted_quantile)

    # # 计算 Fund_return 列与 Adjusted_Quantile 列的相关性
    correlation1 = future_return['Quantile'].corr(future_return['alpha'],method='spearman')
    # fund_return['Correlation'] = None

    # # 计算每个基金的相关性
    # for fund_code in future_return['FundCode']:
    #     other_funds = future_return[future_return['FundCode'] != fund_code]
    #     correlation = other_funds['Quantile'].corr(other_funds['alpha'])
    #     future_return.loc[future_return['FundCode'] == fund_code, 'Correlation'] = correlation

    # future_return_cleaned=future_return.dropna(axis=0)

    return future_return,correlation1

In [14]:
#封装所有
def calculate_rankic_general(start_date,end_date,enddate_str,test_start_time,test_end_time):
    ashare=ashareeodprice()
    ashare.read_data()
    stock_return=ashare.calculate_returns_for_all_stocks(start_date, end_date)
    stock_return_rank_all, top_50_return, bottom_50_return = ashare.rank_and_split_returns(stock_return)

    fundportfolio_calc=fundportfolio()
    fundportfolio_calc.read_data()
    top_holdings=fundportfolio_calc.filter_and_top_holdings_for_all_funds(enddate_str, top_n=10)
    top_holdings_matrix=fundportfolio_calc.create_rank_matrix(top_holdings, top_50_return, bottom_50_return)

    result=calculatestuct(stock_return_rank_all,top_holdings_matrix)

    return_for_fund=fundreturns()
    return_for_fund.get_data()
    fund_return=return_for_fund.calculate_return_rank_for_funds(start_date,end_date)

    final=calculate_alpha(fund_return,result)

    future_return = return_for_fund.calculate_return_rank_for_funds(test_start_time,test_end_time)
    
    plot,rankic=calculate_rankic(result,future_return)

    return final,rankic,plot

In [15]:
# ashare=ashareeodprice()
# ashare.read_data()
# stock_return=ashare.calculate_returns_for_all_stocks('2023-03-31','2023-06-30')
# stock_return_rank_all, top_50_return, bottom_50_return = ashare.rank_and_split_returns(stock_return)

# fundportfolio_calc=fundportfolio()
# fundportfolio_calc.read_data()
# top_holdings=fundportfolio_calc.filter_and_top_holdings_for_all_funds('20230630', top_n=10)
# top_holdings_matrix=fundportfolio_calc.create_rank_matrix(top_holdings, top_50_return, bottom_50_return)

# result=calculatestuct(stock_return_rank_all,top_holdings_matrix)

# return_for_fund=fundreturns()
# return_for_fund.get_data()
# fund_return=return_for_fund.calculate_return_rank_for_funds('2023-03-31','2023-06-30')

# final=calculate_alpha(fund_return,result)

# future_return = return_for_fund.calculate_return_rank_for_funds('2023-06-30','2023-09-30')

# plot,rankic=calculate_rankic(result,future_return)

In [16]:
#绘制柱状图
def barchart(data):
    grouped_data = data.groupby('Fund_return')['alpha'].mean().reset_index()

    # 绘制条形图
    plt.figure(figsize=(10, 6))
    plt.bar(grouped_data['Fund_return'], grouped_data['alpha'], color='b')

    plt.title('Fund_return vs Alpha')
    plt.xlabel('Fund_return')
    plt.ylabel('Average Alpha')
    plt.xticks(rotation=0)
    plt.show()

In [17]:
fund_return_731=fundreturns()
fund_return_731.get_data()
ddd=fund_return_731.calculate_return_rank_for_funds('2023-06-30','2023-07-31')

In [18]:
ddd[ddd['FundCode']=='000001.OF']

,FundCode,Fund_return,Quantile
1739,000001.OF,-0.049041,0.093589


In [19]:
result=calculatestuct(stock_return_rank,b)
result

S_INFO_WINDCODE,000001.OF,000003.OF,000004.OF,000006.OF,000008.OF,000011.OF,000014.OF,000017.OF,000020.OF,000021.OF,...,970130.OF,970131.OF,970184.OF,970185.OF,970188.OF,970189.OF,970190.OF,970204.OF,970205.OF,970207.OF
S_INFO_STOCKWINDCODE,,,,,,,,,,,,,,,,,,,,,
000656.SZ,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
002146.SZ,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
300169.SZ,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
601099.SH,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
600266.SH,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688498.SH,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
603042.SH,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
688700.SH,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192


In [20]:
final=calculate_alpha(ddd,result)
final

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/587150496.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_rank_quantile['Adjusted_Quantile'] = return_rank_quantile.apply(


S_INFO_WINDCODE,000001.OF,000003.OF,000004.OF,000006.OF,000008.OF,000011.OF,000014.OF,000017.OF,000020.OF,000021.OF,...,970130.OF,970131.OF,970184.OF,970185.OF,970188.OF,970189.OF,970190.OF,970204.OF,970205.OF,970207.OF
S_INFO_STOCKWINDCODE,,,,,,,,,,,,,,,,,,,,,
000656.SZ,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
002146.SZ,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
300169.SZ,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
601099.SH,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
600266.SH,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603042.SH,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
688700.SH,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
stuct_raw,0.263091,0.263924,0.263924,0.262936,0.263820,0.264259,0.264014,0.263765,0.263531,0.263203,...,0.264130,0.264130,0.264541,0.264541,0.264954,0.264954,0.264954,0.264687,0.264687,0.264419


## 进行测试

In [21]:
invisible_result_q1,invisible_rankic_q1,invisible_plot_q1=calculate_rankic_general(
    start_date ='2023-03-31',
    end_date = '2023-04-30', 
    enddate_str = '2023-03-31',
    test_start_time='2023-04-30',
    test_end_time='2023-07-31')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [22]:
invisible_result_q2,invisible_rankic_q2,invisible_plot_q2=calculate_rankic_general(
    start_date ='2023-06-30',
    end_date = '2023-07-31', 
    enddate_str = '2023-06-30',
    test_start_time='2023-07-31',
    test_end_time='2023-10-31')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [23]:
invisible_result_q3,invisible_rankic_q3,invisible_plot_q3=calculate_rankic_general(
    start_date ='2023-09-30',
    end_date = '2023-10-31', 
    enddate_str = '2023-09-30',
    test_start_time='2023-10-31',
    test_end_time='2024-01-31')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [24]:
invisible_result_q4,invisible_rankic_q4,invisible_plot_q4=calculate_rankic_general(
    start_date ='2023-12-31',
    end_date = '2024-01-31', 
    enddate_str = '2023-12-31',
    test_start_time='2024-01-31',
    test_end_time='2024-04-30')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [25]:
invisible=pd.DataFrame([invisible_rankic_q1,invisible_rankic_q2,invisible_rankic_q3,invisible_rankic_q4])

In [26]:
invisible

,0
0,-0.146676
1,0.075489
2,0.168525
3,-0.515669


In [27]:
icir_invisible=invisible.mean()/invisible.std()
icir_invisible

0   -0.343694
dtype: float64

In [28]:
# def calculate_periodic_rankic(start_date, end_date, output_csv):
#     # Initialize results
#     rankic_results = []

#     # Convert strings to datetime
#     start_date = datetime.strptime(start_date, '%Y-%m-%d')
#     end_date = datetime.strptime(end_date, '%Y-%m-%d')

#     # Loop through each quarter
#     current_start = start_date
#     while current_start < end_date:
#         current_end = current_start + relativedelta(months=3) - timedelta(days=1)
#         test_start_time = current_end
#         test_end_time = test_start_time + relativedelta(months=3) - timedelta(days=1)
        
#         # Ensure dates are within the provided range
#         if test_end_time > end_date:
#             test_end_time = end_date
        
#         # Convert dates to string format
#         start_str = current_start.strftime('%Y-%m-%d')
#         end_str = current_end.strftime('%Y-%m-%d')
#         test_start_str = test_start_time.strftime('%Y-%m-%d')
#         test_end_str = test_end_time.strftime('%Y-%m-%d')

#         # Calculate rank IC for the current period
#         final, rankic, plot = calculate_rankic_general(
#             start_str, end_str, end_str, test_start_str, test_end_str
#         )
        
#         # Store the results
#         rankic_results.append({
#             'start_date': start_str,
#             'end_date': end_str,
#             'test_start_date': test_start_str,
#             'test_end_date':test_end_str,
#             'rankic': rankic
#         })
        
#         # Move to the next quarter
#         current_start = current_end + timedelta(days=1)

#     # Convert the results to a DataFrame
#     rankic_df = pd.DataFrame(rankic_results)

#     # Write the results to a CSV file
#     rankic_df.to_csv(output_csv, index=False)

#     return rankic_results

In [29]:
dressing_result_q1,dressing_rankic_q1,dressing_plot_q1=calculate_rankic_general(
    start_date ='2023-01-01',
    end_date = '2023-03-31', 
    enddate_str = '2023-03-31',
    test_start_time='2023-03-31',
    test_end_time='2023-06-30')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [30]:
dressing_result_q2,dressing_rankic_q2,dressing_plot_q2=calculate_rankic_general(
    start_date ='2023-03-31',
    end_date = '2023-06-30', 
    enddate_str = '2023-06-30',
    test_start_time='2023-06-30',
    test_end_time='2023-09-30')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [31]:
dressing_result_q3,dressing_rankic_q3,dressing_plot_q3=calculate_rankic_general(
    start_date ='2023-06-30',
    end_date = '2023-09-30', 
    enddate_str = '2023-09-30',
    test_start_time='2023-09-30',
    test_end_time='2023-12-31')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [32]:
dressing_result_q4,dressing_rankic_q4,dressing_plot_q4=calculate_rankic_general(
    start_date ='2023-09-30',
    end_date = '2023-12-31', 
    enddate_str = '2023-12-31',
    test_start_time='2023-12-31',
    test_end_time='2024-03-30')

/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Rank'] = top_50['Return'].rank(ascending=True)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_50['Rank'] = bottom_50['Return'].rank(ascending=False)
/var/folders/fj/qfgcy6_x669fxrp49s5jltzh0000gn/T/ipykernel_62344/1866825925.py:63: SettingWithCopyWarning: 
A value is trying to be set o

In [33]:
dressing=pd.DataFrame([dressing_rankic_q1,dressing_rankic_q2,dressing_rankic_q3,dressing_rankic_q4])
ciir=dressing.mean()/dressing.std()
ciir

0    1.994625
dtype: float64

In [34]:
dressing

,0
0,0.239578
1,0.303255
2,0.238979
3,0.058723


In [35]:
dressing_result_q2

S_INFO_WINDCODE,000001.OF,000003.OF,000004.OF,000006.OF,000008.OF,000011.OF,000014.OF,000017.OF,000020.OF,000021.OF,...,970130.OF,970131.OF,970184.OF,970185.OF,970188.OF,970189.OF,970190.OF,970204.OF,970205.OF,970207.OF
S_INFO_STOCKWINDCODE,,,,,,,,,,,,,,,,,,,,,
301205.SZ,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
872808.BJ,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
300678.SZ,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
688498.SH,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,...,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190,0.000190
300308.SZ,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,...,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174,0.000174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688039.SH,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
001367.SZ,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,...,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
stuct_raw,0.263188,0.262501,0.262501,0.264168,0.262615,0.262678,0.262388,0.263019,0.263110,0.263366,...,0.262545,0.262545,0.263533,0.263533,0.262683,0.262683,0.262683,0.262610,0.262610,0.262528


In [36]:
dressing_alpha=pd.DataFrame(dressing_result_q2.iloc[-1,:])
dressing_alpha=dressing_alpha.rename(columns={'alpha':'dressing'})
invisible_alpha=pd.DataFrame(invisible_result_q2.iloc[-1,:])
invisible_alpha=invisible_alpha.rename(columns={'alpha':'invisible'})
matching=pd.concat([dressing_alpha,invisible_alpha],axis=1)
matching=matching.dropna(axis=0)
matching

,dressing,invisible
S_INFO_WINDCODE,,
000001.OF,-0.029893,-0.179432
000003.OF,-0.004913,0.224511
000004.OF,-0.005720,0.213054
000006.OF,0.038456,0.409649
000008.OF,-0.201718,0.301452
...,...,...
970189.OF,0.287669,-0.099276
970190.OF,0.247403,-0.105139
970204.OF,0.129517,-0.077075


In [37]:
def standardize(df):
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        lower_limit = mean - 3 * std
        upper_limit = mean + 3 * std
        df[col] = df[col].apply(lambda x: lower_limit if x < lower_limit else (upper_limit if x > upper_limit else x))

          # 标准化处理
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        df[col] = (df[col] - mean) / std
    return df

In [38]:
def calculate_matching(dressing_result,invisible_result,dressing_plot):
    dressing_alpha=pd.DataFrame(dressing_result.iloc[-1,:])
    dressing_alpha=dressing_alpha.rename(columns={'alpha':'dressing'})
    invisible_alpha=pd.DataFrame(invisible_result.iloc[-1,:])
    invisible_alpha=invisible_alpha.rename(columns={'alpha':'invisible'})
    matching=pd.concat([dressing_alpha,invisible_alpha],axis=1)
    matching=matching.dropna(axis=0)
    standardize(matching)
    matching['matching']=(1/2*matching['dressing']+1/2*matching['invisible'])
    merged_df = pd.merge(dressing_plot, matching, left_on='FundCode', right_index=True)
    matching_rankic=merged_df['matching'].corr(merged_df['Quantile'],method='spearman')
    return matching_rankic

In [39]:
calculate_matching(dressing_result_q1,invisible_result_q1,invisible_plot_q1)

-0.08746576885259875

In [40]:
calculate_matching(dressing_result_q2,invisible_result_q2,invisible_plot_q2)

0.24649216570786317

In [41]:
calculate_matching(dressing_result_q3,invisible_result_q3,invisible_plot_q3)

0.33105663031329635

In [42]:
calculate_matching(dressing_result_q4,invisible_result_q4,invisible_plot_q4)

-0.5663233278759497